# **Aprendendo a construir uma algoritmo de Recomendação**




1.   Importar as Bibliotecas que serão utilizadas
2.   Coleta e pré-processamento dos dados: Importar o dataset e separar as colunas com informações úteis




In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

filmes = pd.read_csv('/content/movies.csv')
filmes.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [ ]:
filmes.shape

(4803, 24)

Definir as colunas relevantes para a recomendação

In [ ]:
colunas_relevantes = ['genres', 'keywords', 'tagline', 'cast', 'director']
#Trocar as células vazias por strings vazias
for coluna in colunas_relevantes:
  filmes[coluna] = filmes[coluna].fillna('')

Combinar as Colunas relevantes

In [ ]:
colunas_combinadas = filmes['genres'] + ' ' + filmes['keywords'] + ' ' + filmes['tagline'] + ' ' + filmes['cast'] + ' ' + filmes['director']
print(colunas_combinadas)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


# **Processo de Vetorização**

https://medium.com/data-hackers/tf-idf-algoritmo-de-recomenda%C3%A7%C3%A3o-6c3cbd55e439

Transforma um conjunto de documentos de texto em vetores numéricos, com base na importância das palavras em cada documento e no conjunto como um todo.
Essa importância é medida por meio do conceito de TF-IDF:



1.   **TF** : É a frequência de uma palavra em um documento =>

  TF(t,d) = número de vezes que o termo 't' aparece no documento 'd' / total de termos no documento 'd'  


2.   **IDF** : idf é uma medida de quão comum ou raro um termo é em todo os documentos. Portanto, o ponto a ser observado é que é comum a todos os documentos =>

  IDF(t) = log(número total de documentos / número de documentos que contém 't')


3.   **TF-IDF(t,d) = TF(t,d) * IDF(t)**





In [ ]:
vetor = TfidfVectorizer()
vetor_dados = vetor.fit_transform(colunas_combinadas)
print(vetor_dados)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 124266 stored elements and shape (4803, 17318)>
  Coords	Values
  (0, 201)	0.07860022416510505
  (0, 274)	0.09021200873707368
  (0, 5274)	0.11108562744414445
  (0, 13599)	0.1036413987316636
  (0, 5437)	0.1036413987316636
  (0, 3678)	0.21392179219912877
  (0, 3065)	0.22208377802661425
  (0, 5836)	0.1646750903586285
  (0, 14378)	0.33962752210959823
  (0, 16587)	0.12549432354918996
  (0, 3225)	0.24960162956997736
  (0, 14271)	0.21392179219912877
  (0, 4945)	0.24025852494110758
  (0, 15261)	0.07095833561276566
  (0, 16998)	0.1282126322850579
  (0, 11192)	0.09049319826481456
  (0, 11503)	0.27211310056983656
  (0, 13349)	0.15021264094167086
  (0, 17007)	0.23643326319898797
  (0, 17290)	0.20197912553916567
  (0, 13319)	0.2177470539412484
  (0, 14064)	0.20596090415084142
  (0, 16668)	0.19843263965100372
  (0, 14608)	0.15150672398763912
  (0, 8756)	0.22709015857011816
  :	:
  (4801, 403)	0.17727585190343229
  (4801, 4835)	0.247137650

# **Similaridade dos Cossenos**
https://www.geeksforgeeks.org/cosine-similarity/

A similaridade de cosseno é uma métrica útil para determinar a semelhança dos objetos de dados, independentemente de seu tamanho.

A similaridade de cosseno mede a semelhança entre dois vetores diferentes de zero calculando o cosseno do ângulo entre eles.

A fórmula para encontrar a semelhança de cosseno entre dois vetores é:

**cosine_similarity(x, y) = x * y / ||x|| * ||y||**

In [ ]:
similaridade = cosine_similarity(vetor_dados)
print(similaridade)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [ ]:
print(similaridade.shape)

(4803, 4803)


-------------------

In [ ]:
filme = input(' Escolha seu filme favorito : ')

#Listando todos os títulos de filmes do dataset
lista_titulos = filmes['title'].tolist()

#encontrando os títulos (3) mais semelhante ao input do usuário
encontrar_semelhante = difflib.get_close_matches(filme, lista_titulos)

#Selecionando o título MAIS semelhante ao input
o_mais_semelhante = encontrar_semelhante[0]

#Encontrando o ID do filme selecionado
filme_id = filmes[filmes.title == o_mais_semelhante]['index'].values[0]

#Lista dos filmes similares
pontuacao_similaridade = list(enumerate(similaridade[filme_id]))

#Ordenando os filmes na ordem de sua pontuação de similaridade
filmes_similares_ordenados = sorted(pontuacao_similaridade, key = lambda x:x[1], reverse = True)

print('Sugestões de Filmes para você : \n')

i = 1
#Printando os filmes a partir do ID
for movie in filmes_similares_ordenados:
  id = movie[0]
  titulo_do_id = filmes[filmes.index==id]['title'].values[0]
  if (i<30):
    print(i, '.',titulo_do_id)
    i+=1

 Escolha seu filme favorito : Avengers
Sugestões de Filmes para você : 

1 . The Avengers
2 . Avengers: Age of Ultron
3 . Captain America: The Winter Soldier
4 . Captain America: Civil War
5 . Iron Man 2
6 . Thor: The Dark World
7 . X-Men
8 . The Incredible Hulk
9 . X-Men: Apocalypse
10 . Ant-Man
11 . Thor
12 . X2
13 . X-Men: The Last Stand
14 . Deadpool
15 . X-Men: Days of Future Past
16 . Captain America: The First Avenger
17 . The Amazing Spider-Man 2
18 . The Image Revolution
19 . Iron Man
20 . Iron Man 3
21 . Man of Steel
22 . The Spirit
23 . Superman II
24 . X-Men: First Class
25 . Guardians of the Galaxy
26 . Batman v Superman: Dawn of Justice
27 . Serenity
28 . Spawn
29 . Teenage Mutant Ninja Turtles: Out of the Shadows
